# Question 2
# *Which genres of film have the highest ratings?* 

---

## 2.1 - Importing the Required Modules and DataFrames

### 2.1.1 - Import the Required Modules

In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns
sns.set_style("whitegrid")
sns.set_context("talk")

### 2.1.2 - Load the Required Mapping DataFrames

In [2]:
id_to_genre_map = pd.read_csv("maps/id_to_genre_map.csv", index_col="movie_ID")
titleyear_to_id_map = pd.read_csv("maps/titleyear_to_id_map.csv", index_col=["title", "year"])

In [3]:
id_to_genre_map.head()

,genre_1,genre_2,genre_3
movie_ID,,,
tt0063540,Action,Crime,Drama
tt0066787,Biography,Drama,NaN
tt0069049,Drama,NaN,NaN
tt0069204,Comedy,Drama,NaN
tt0100275,Comedy,Drama,Fantasy


In [4]:
titleyear_to_id_map.head()

,,movie_ID
title,year,
Sunghursh,2013,tt0063540
One Day Before the Rainy Season,2019,tt0066787
The Other Side of the Wind,2018,tt0069049
Sabse Bada Sukh,2018,tt0069204
The Wandering Soap Opera,2017,tt0100275


##
---

## 2.2 - Importing and Cleaning the Dataset in `tmdb.movies.csv`

### 2.2.1 - Import the Dataset in `tmdb.movies.csv`

In [5]:
tmdb_movies_df = pd.read_csv("data/tmdb.movies.csv")

In [6]:
tmdb_movies_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26517 entries, 0 to 26516
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Unnamed: 0         26517 non-null  int64  
 1   genre_ids          26517 non-null  object 
 2   id                 26517 non-null  int64  
 3   original_language  26517 non-null  object 
 4   original_title     26517 non-null  object 
 5   popularity         26517 non-null  float64
 6   release_date       26517 non-null  object 
 7   title              26517 non-null  object 
 8   vote_average       26517 non-null  float64
 9   vote_count         26517 non-null  int64  
dtypes: float64(2), int64(3), object(5)
memory usage: 2.0+ MB


### 2.2.2 - Drop the Unnecessary Columns from `tmdb.movies.csv`

In [7]:
tmdb_movies_df.drop(["Unnamed: 0", "genre_ids", "id", "original_language", "original_title", "popularity"], axis=1, inplace=True)

In [8]:
tmdb_movies_df.columns

Index(['release_date', 'title', 'vote_average', 'vote_count'], dtype='object')

### 2.2.3 - Discard the Day and Month of each Date in the `release_date` Column

In [9]:
tmdb_movies_df["release_date"].head()

0    2010-11-19
1    2010-03-26
2    2010-05-07
3    1995-11-22
4    2010-07-16
Name: release_date, dtype: object

In [10]:
tmdb_movies_df["release_date"] = tmdb_movies_df["release_date"].str.split('-').map(lambda x: int(x[0]))

In [11]:
tmdb_movies_df["release_date"].head()

0    2010
1    2010
2    2010
3    1995
4    2010
Name: release_date, dtype: int64

### 2.2.4 - Rename the Relevant Columns of `tmdb.movies.csv`

In [12]:
tmdb_movies_df.rename(columns={"release_date": "year", "vote_average": "average_rating"}, inplace=True)

In [13]:
tmdb_movies_df.columns

Index(['year', 'title', 'average_rating', 'vote_count'], dtype='object')

### 2.2.5 - Drop Ratings wich have an Unnacceptably Small Number of Votes and then Discard the `vote_count` Column

In [14]:
tmdb_movies_df = tmdb_movies_df[tmdb_movies_df["vote_count"] >= 10]
tmdb_movies_df.drop(columns="vote_count", inplace=True)

In [15]:
tmdb_movies_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10397 entries, 0 to 26464
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   year            10397 non-null  int64  
 1   title           10397 non-null  object 
 2   average_rating  10397 non-null  float64
dtypes: float64(1), int64(1), object(1)
memory usage: 324.9+ KB


### 2.2.6 - Set the `title` and `year` Columns as the Index of `tmdb_movies_df`

In [16]:
tmdb_movies_df.set_index(["title", "year"], inplace=True)

In [17]:
tmdb_movies_df.head()

,,average_rating
title,year,
Harry Potter and the Deathly Hallows: Part 1,2010,7.7
How to Train Your Dragon,2010,7.7
Iron Man 2,2010,6.8
Toy Story,1995,7.9
Inception,2010,8.3


### 2.2.7 - Append the Column with the Associated *Movie ID*  for each *Title* and *Year of Release* by Joining `titleyear_to_id_map` with `tmdb_movies_df`

In [18]:
tmdb_movies_df = tmdb_movies_df.join(titleyear_to_id_map, how='inner')

In [ ]:
tmdb_movies_df.head()

,,average_rating,movie_ID
title,year,,
#Horror,2015,3.3,tt3526286
#SquadGoals,2018,4.3,tt6540984
#Stuck,2014,5.3,tt2075318
$50K and a Call Girl: A Love Story,2014,6.8,tt2106284
(A)sexual,2011,5.3,tt1811293


### 2.2.8 - Set the `movie_ID` Column as the Index of `tmdb_movies_df`

In [ ]:
tmdb_movies_df.set_index("movie_ID", inplace=True)

In [ ]:
tmdb_movies_df.head()

,average_rating
movie_ID,
tt3526286,3.3
tt6540984,4.3
tt2075318,5.3
tt2106284,6.8
tt1811293,5.3


### 2.2.9 - Export `tmdb_movies_df` [Needed for Question 3]

In [ ]:
tmdb_movies_df.to_csv("maps/id_to_tmdb_rating_map.csv")

##
---

## 2.3 - Creating a Dataframe (`tmdb_rating_by_genre_df`) for the *TMDb Average Rating* by Genre

### 2.3.1 - Initialize `tmdb_rating_by_genre_df` by Appending Columns with the Associated *Genres* for each *Movie ID* 

In [ ]:
tmdb_rating_by_genre_df = tmdb_movies_df.join(id_to_genre_map, how='inner')

In [ ]:
tmdb_rating_by_genre_df.head()

,average_rating,genre_1,genre_2,genre_3
movie_ID,,,,
tt0069049,7.0,Drama,NaN,NaN
tt0315642,6.6,Action,Crime,Drama
tt0326965,5.4,Drama,Mystery,Thriller
tt0337692,5.6,Adventure,Drama,Romance
tt0359950,7.1,Adventure,Comedy,Drama


### 2.3.2 - Convert `tmdb_rating_by_genre_df` to Long Format

In [ ]:
tmdb_rating_by_genre_df = pd.melt(tmdb_rating_by_genre_df.reset_index(), id_vars=["movie_ID", "average_rating"])
tmdb_rating_by_genre_df.set_index("movie_ID", inplace=True)

In [ ]:
tmdb_rating_by_genre_df.head(10)

,average_rating,variable,value
movie_ID,,,
tt0069049,7.0,genre_1,Drama
tt0315642,6.6,genre_1,Action
tt0326965,5.4,genre_1,Drama
tt0337692,5.6,genre_1,Adventure
tt0359950,7.1,genre_1,Adventure
tt0360556,5.4,genre_1,Drama
tt0365545,7.3,genre_1,Comedy
tt0365907,6.3,genre_1,Action
tt0369610,6.6,genre_1,Action


### 2.3.3 - Drop the `variable` Column and Rename the `value` Column to `genre`

In [ ]:
tmdb_rating_by_genre_df.drop(columns="variable", inplace=True)
tmdb_rating_by_genre_df.rename(columns={"value": "genre"}, inplace=True)

In [ ]:
tmdb_rating_by_genre_df.head(10)

,average_rating,genre
movie_ID,,
tt0069049,7.0,Drama
tt0315642,6.6,Action
tt0326965,5.4,Drama
tt0337692,5.6,Adventure
tt0359950,7.1,Adventure
tt0360556,5.4,Drama
tt0365545,7.3,Comedy
tt0365907,6.3,Action
tt0369610,6.6,Action


### 2.3.4 - Drop the Rows with a Null Value in the `genre` Column

In [ ]:
tmdb_rating_by_genre_df.info() 

<class 'pandas.core.frame.DataFrame'>
Index: 18639 entries, tt0069049 to tt9889072
Data columns (total 2 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   average_rating  18639 non-null  float64
 1   genre           14068 non-null  object 
dtypes: float64(1), object(1)
memory usage: 436.9+ KB


In [ ]:
tmdb_rating_by_genre_df.dropna(inplace=True)

In [ ]:
tmdb_rating_by_genre_df.info() 

<class 'pandas.core.frame.DataFrame'>
Index: 14068 entries, tt0069049 to tt9495224
Data columns (total 2 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   average_rating  14068 non-null  float64
 1   genre           14068 non-null  object 
dtypes: float64(1), object(1)
memory usage: 329.7+ KB


### 2.3.5 - Group `tmdb_rating_by_genre_df` by *Genre* and View the Number of Data Points in each Group

In [ ]:
tmdb_genre_counts = tmdb_rating_by_genre_df.groupby("genre").count()["average_rating"].sort_values(ascending=False)

In [ ]:
tmdb_genre_counts

genre
Drama          2941
Comedy         1740
Thriller       1222
Action         1190
Horror         1082
Crime           816
Romance         733
Adventure       729
Documentary     720
Mystery         478
Biography       476
Sci-Fi          418
Fantasy         338
Animation       239
Family          237
History         202
Music           180
Sport           141
War              78
Musical          43
Western          39
News             24
Game-Show         1
Reality-TV        1
Name: average_rating, dtype: int64

### 2.3.6 - Drop Genres with an Unacceptably Small Number of Data Points

In [ ]:
tmdb_acceptable_genres = tmdb_genre_counts.loc[lambda x: x>= 100].index

In [ ]:
tmdb_acceptable_genres

Index(['Drama', 'Comedy', 'Thriller', 'Action', 'Horror', 'Crime', 'Romance',
       'Adventure', 'Documentary', 'Mystery', 'Biography', 'Sci-Fi', 'Fantasy',
       'Animation', 'Family', 'History', 'Music', 'Sport'],
      dtype='object', name='genre')

In [ ]:
tmdb_rating_by_genre_df = tmdb_rating_by_genre_df.loc[tmdb_rating_by_genre_df["genre"].map(lambda x: x in tmdb_acceptable_genres)]

In [ ]:
set(tmdb_rating_by_genre_df["genre"].unique()) == set(tmdb_acceptable_genres)

True

### 2.3.7 - Obtain the Mean Value of `average_rating` for Each Genre

In [ ]:
tmdb_rating_by_genre_df = tmdb_rating_by_genre_df.groupby("genre").mean()["average_rating"].sort_values(ascending=False)

In [ ]:
tmdb_rating_by_genre_df

genre
Documentary    6.947222
Biography      6.859664
Music          6.786667
History        6.701980
Sport          6.634043
Animation      6.509623
Romance        6.204638
Drama          6.201836
Adventure      6.054458
Family         6.011814
Crime          5.980515
Comedy         5.906092
Fantasy        5.826627
Mystery        5.768828
Action         5.756807
Sci-Fi         5.544737
Thriller       5.468576
Horror         5.053512
Name: average_rating, dtype: float64

##
---

## 2.4 - Importing and Cleaning the Dataset in `imdb.title.ratings.csv`

### 2.4.1 - Import the Dataset in `imdb.title.ratings.csv`

In [ ]:
imdb_title_ratings_df = pd.read_csv("data/imdb.title.ratings.csv")

In [ ]:
imdb_title_ratings_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 73856 entries, 0 to 73855
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   tconst         73856 non-null  object 
 1   averagerating  73856 non-null  float64
 2   numvotes       73856 non-null  int64  
dtypes: float64(1), int64(1), object(1)
memory usage: 1.7+ MB


### 2.4.2 - Rename the Relevant Columns of `imdb_title_ratings_df`

In [ ]:
imdb_title_ratings_df.rename(columns={"tconst": "movie_ID", "averagerating": "average_rating", "numvotes": "vote_count"}, inplace=True)

In [ ]:
imdb_title_ratings_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 73856 entries, 0 to 73855
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   movie_ID        73856 non-null  object 
 1   average_rating  73856 non-null  float64
 2   vote_count      73856 non-null  int64  
dtypes: float64(1), int64(1), object(1)
memory usage: 1.7+ MB


### 2.4.3 - Drop Ratings wich have an Unnacceptably Small Number of Votes and then Discard the `vote_count` Column

In [ ]:
imdb_title_ratings_df = imdb_title_ratings_df[imdb_title_ratings_df["vote_count"] >= 10]
imdb_title_ratings_df.drop(columns="vote_count", inplace=True)

In [ ]:
imdb_title_ratings_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 61710 entries, 0 to 73855
Data columns (total 2 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   movie_ID        61710 non-null  object 
 1   average_rating  61710 non-null  float64
dtypes: float64(1), object(1)
memory usage: 1.4+ MB


### 2.4.4 - Set the `movie_ID` Column as the Index of `imdb_title_ratings_df`

In [ ]:
imdb_title_ratings_df.set_index("movie_ID", inplace=True)

In [ ]:
imdb_title_ratings_df.head()

,average_rating
movie_ID,
tt10356526,8.3
tt10384606,8.9
tt1042974,6.4
tt1043726,4.2
tt1060240,6.5


### 2.4.5 - Export `imdb_title_ratings_df` [Needed for Question 3]

In [ ]:
imdb_title_ratings_df.to_csv("maps/id_to_imdb_rating_map.csv")

##
---

## 2.5 - Creating a Dataframe (`imdb_rating_by_genre_df`) for the *IMDb Average Rating* by Genre

### 2.5.1 - Initialize `imdb_rating_by_genre_df` by Appending Columns with the Associated *Genres* for each *Movie ID* 

In [ ]:
imdb_rating_by_genre_df = imdb_title_ratings_df.join(id_to_genre_map, how='inner')

In [ ]:
imdb_rating_by_genre_df.head()

,average_rating,genre_1,genre_2,genre_3
movie_ID,,,,
tt10356526,8.3,Romance,NaN,NaN
tt10384606,8.9,Documentary,NaN,NaN
tt1042974,6.4,Drama,NaN,NaN
tt1043726,4.2,Action,Adventure,Fantasy
tt1060240,6.5,Mystery,Thriller,NaN


### 2.5.2 - Convert `imdb_rating_by_genre_df` to Long Format

In [ ]:
imdb_rating_by_genre_df = pd.melt(imdb_rating_by_genre_df.reset_index(), id_vars=["movie_ID", "average_rating"])
imdb_rating_by_genre_df.set_index("movie_ID", inplace=True)

In [ ]:
imdb_rating_by_genre_df.head(10)

,average_rating,variable,value
movie_ID,,,
tt10356526,8.3,genre_1,Romance
tt10384606,8.9,genre_1,Documentary
tt1042974,6.4,genre_1,Drama
tt1043726,4.2,genre_1,Action
tt1060240,6.5,genre_1,Mystery
tt1069246,6.2,genre_1,Comedy
tt1094666,7.0,genre_1,Biography
tt1130982,6.4,genre_1,Drama
tt1156528,7.2,genre_1,Drama


### 2.5.3 - Drop the `variable` Column and Rename the `value` Column to `genre`

In [ ]:
imdb_rating_by_genre_df.drop(columns="variable", inplace=True)
imdb_rating_by_genre_df.rename(columns={"value": "genre"}, inplace=True)

In [ ]:
imdb_rating_by_genre_df.head(10)

,average_rating,genre
movie_ID,,
tt10356526,8.3,Romance
tt10384606,8.9,Documentary
tt1042974,6.4,Drama
tt1043726,4.2,Action
tt1060240,6.5,Mystery
tt1069246,6.2,Comedy
tt1094666,7.0,Biography
tt1130982,6.4,Drama
tt1156528,7.2,Drama


### 2.5.4 - Drop the Rows with a Null Value in the `genre` Column

In [ ]:
imdb_rating_by_genre_df.dropna(inplace=True)

In [ ]:
imdb_rating_by_genre_df.info() 

<class 'pandas.core.frame.DataFrame'>
Index: 110029 entries, tt10356526 to tt9844256
Data columns (total 2 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   average_rating  110029 non-null  float64
 1   genre           110029 non-null  object 
dtypes: float64(1), object(1)
memory usage: 2.5+ MB


### 2.5.5 - Group `imdb_rating_by_genre_df` by *Genre* and View the Number of Data Points for each Group

In [ ]:
genre_counts = imdb_rating_by_genre_df.groupby("genre").count()["average_rating"].sort_values(ascending=False)

In [ ]:
genre_counts

genre
Drama          26761
Comedy         15436
Documentary    12850
Thriller        7440
Horror          6842
Action          6262
Romance         5941
Crime           4159
Adventure       3361
Biography       3089
Family          2891
Mystery         2703
History         2260
Sci-Fi          1980
Fantasy         1872
Music           1625
Animation       1571
Sport            987
War              717
Musical          587
News             436
Western          244
Reality-TV        12
Adult              2
Game-Show          1
Name: average_rating, dtype: int64

### 2.5.6 - Drop Genres that Are Not In `tmdb_acceptable_genres`

In [ ]:
imdb_rating_by_genre_df = imdb_rating_by_genre_df.loc[imdb_rating_by_genre_df["genre"].map(lambda x: x in tmdb_acceptable_genres)]

In [ ]:
set(imdb_rating_by_genre_df["genre"].unique()) == set(tmdb_acceptable_genres)

True

### 2.5.7 - Obtain the Mean Value of `average_rating` for Each Genre

In [ ]:
imdb_rating_by_genre_df = imdb_rating_by_genre_df.groupby("genre").mean()["average_rating"].sort_values(ascending=False)

In [ ]:
imdb_rating_by_genre_df

genre
Documentary    7.303751
Biography      7.107737
Music          7.020369
History        6.977788
Sport          6.883587
Drama          6.342678
Family         6.302871
Animation      6.243666
Romance        6.110066
Adventure      6.098631
Crime          6.065497
Comedy         5.925434
Mystery        5.837551
Fantasy        5.828205
Action         5.739604
Thriller       5.573374
Sci-Fi         5.404949
Horror         4.934142
Name: average_rating, dtype: float64

##
---

## 2.6 - Visualizing the Correlation between *Genre* and *TMDb Average Rating* 

In [ ]:
df = tmdb_rating_by_genre_df
sns.set_palette('dark')
fig, ax = plt.subplots(figsize=(20,8))
ax = sns.barplot(x=df.values, 
                 y=df.index,
                 color='mediumspringgreen',
                 orient="h",
                 dodge=False)
ax.set_xlim(0,10)
ax.set_xlabel("Rating")
ax.set_ylabel("Genre")
ax.set_title("Mean $TMBd$ Rating by Genre")


Text(0.5, 1.0, 'Mean $TMBd$ Rating by Genre')

##
---

## 2.7 - Visualizing the Correlation between *Genre* and *IMDb Average Rating* 

In [ ]:
df = imdb_rating_by_genre_df

fig, ax = plt.subplots(figsize=(20,8))
ax = sns.barplot(x=df.values, 
                 y=df.index,
                 color='tomato',
                 orient="h",
                 dodge=False)
ax.set_xlim(0,10)
ax.set_xlabel("Rating")
ax.set_ylabel("Genre")
ax.set_title("Mean $IMBd$ Rating by Genre")

Text(0.5, 1.0, 'Mean $IMBd$ Rating by Genre')

##
---